# Machine Learning Operations (MLOps) & MLflow demo

## What is MLOps?

MLOps takes a software engineering approach to machine learning — from training the machine learning models to deploying them into production. MLOps is a collection of best practices to increase the value from machine learning through systematic improvement of the process.

The term MLOps is the combination of two fields ML (Machine Learning) and Ops (Operations) – which may refer to development/system operations. MLOps is a systematic process for developing, training, deploying, and optimizing all kinds of machine learning models (e.g., statistical, deep learning, and natural language processing) that enable AI-based applications. 

Organizations are increasingly relying on machine learning (ML) models to drive business value, but the operationalization of these models remains a challenge. Unlike traditional software applications, ML models require special considerations due to their reliance on data, the complexity of the algorithms used, and the need for continual model improvement. Without proper MLOps practices, organizations risk deploying models that are inaccurate, unreliable, or do not scale, leading to negative business outcomes and wasted resources.

The motivation for doing MLOps is to address these challenges and ensure that organizations can deliver high-quality, reliable ML models to production environments. MLOps helps to streamline the ML lifecycle by providing a framework for data management, model training, deployment, and monitoring. It also provides a way to manage the complex infrastructure required to support ML workloads, including specific data storage and compute requirements.

ML models require special considerations compared to traditional software development because they are reliant on data, which can be noisy, incomplete, or biased. This means that data management and preprocessing are critical components of MLOps. In addition, ML models require ongoing improvement and iteration, which requires specialized tooling and processes to support experimentation and model tracking.

Most MLOps platforms offer collaborative model construction, training, assessment, and deployment in addition to data preparation to drive the modeling and training processes.

## MLOps steps

1. **Collect data**: Gather the large quantity of high quality data is often the longest and most expensive step.
<br><br>
1. **Process data**: Most data needs to extensively cleaned before it is ready for ML. Supervised ML is the most powerful type of ML and requires each piece of data have a trustworthy label.
<br><br>
1. **Train and improve models**: Select ML algorithms, then train the model, try different hyperparameters. This steps takes the most amount of compute resources. At a large scale, ML tracking software is useful.
<br><br>
1. **Deploy models to production**: Package trained model and dependencies for a specific production environment.
<br><br>
1. **Monitor models in production**: Once the model is being served to end users, it has to be watched. There are standard software in production requirements. The additional ML requirements are changes in data. The distribution of data could change, there could be new features, and new target labels.
<br><br>
1. **Repeat steps 1 - 5**: ML is iterative process. Each step is an ongoing process that needs change management.

## MLflow

MLflow is an open source platform that is designed to help manage the ML lifecycle.  

Primary MLflow components:

* Tracking: Keeps track of all the runs - code versions, start/end times, metrics, and other information.
* Projects: Provides a standard way to package all of the relevant code.
* Models: Ensures formatting and keeps metadata to enables to be served in production environments
* Model Registry: A centralized model store to help collaboratively manage the full lifecycle of an MLflow model.   

![workflow overview](images/quickstart_tracking_overview.png)
[Image source](https://mlflow.org/)

Let's walk through how to use MLflow to track machine learning experiments

In [2]:
# Load the data
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()

X = data.data
y = data.target

In [3]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
# Define the models and their possible hyperparameter options
from sklearn.ensemble     import RandomForestRegressor
from sklearn.linear_model import Ridge

models = {
    "RidgeRegression": {
        "model": Ridge(),
        "params": {
            "alpha": [0, 1]
        }
    },
    "RandomForestRegression": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": [5, 10],
            "max_depth": [3, 5]
        }
    },
}

In [6]:
# Run cross validation experiments and track results with MLflow
import mlflow
import mlflow.sklearn
import numpy as np
from   mlflow.tracking         import MlflowClient
from   sklearn.model_selection import GridSearchCV, cross_val_score
from   sklearn.metrics         import mean_squared_error

client = MlflowClient()

for model_name, model_info in models.items():
    with mlflow.start_run() as run:
        # Run grid search cross validation
        clf = GridSearchCV(model_info["model"], model_info["params"], cv=3)
        clf.fit(X_train, y_train)

        # Log parameters, metrics, and model
        mlflow.log_param("Model", model_name)
        for param_name, param_value in clf.cv_results_.items():
            mlflow.log_param(param_name, param_value)

        # Cross validation scores
        cv_scores = cross_val_score(clf.best_estimator_, X_train, y_train, cv=3)
        mlflow.log_metric("CV mean score", np.mean(cv_scores))
        mlflow.log_metric("CV std score", np.std(cv_scores))

        # Test score
        test_pred = clf.best_estimator_.predict(X_test)
        test_score = mean_squared_error(y_test, test_pred)
        mlflow.log_metric("Test score", test_score)

        mlflow.sklearn.log_model(clf.best_estimator_, f"{model_name}_model")

        # Get the current run's details using MLflow client
        run_id = run.info.run_id
        run_data = client.get_run(run_id).data

        # Print the results
        print(f"Results for Run ID: {run_id}")
        for param_key, param_value in run_data.params.items():
            print(f"Parameter: {param_key:<17} -> {param_value}")

        for metric_key, metric_value in run_data.metrics.items():
            print(f"Metric:    {metric_key:<17} -> {metric_value}")
        print("-"*30)

/Users/brian/.local/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Results for Run ID: c2bbcf1045004161a05a5bcf94ef554c
Parameter: std_fit_time      -> [3.13553033e-03 5.12214757e-06]
Parameter: mean_test_score   -> [-0.59714628 -0.5974207 ]
Parameter: split0_test_score -> [0.58939904 0.58945712]
Parameter: std_test_score    -> [1.69328141 1.69370267]
Parameter: mean_fit_time     -> [0.0030121  0.00068514]
Parameter: split2_test_score -> [-2.99177553 -2.99264591]
Parameter: split1_test_score -> [0.61093764 0.61092669]
Parameter: param_alpha       -> [0 1]
Parameter: params            -> [{'alpha': 0}, {'alpha': 1}]
Parameter: mean_score_time   -> [0.00020138 0.00014901]
Parameter: Model             -> RidgeRegression
Parameter: std_score_time    -> [5.16374256e-05 7.78671819e-07]
Parameter: rank_test_score   -> [1 2]
Metric:    CV std score      -> 1.6932814109447893
Metric:    Test score        -> 0.5140713616727152
Metric:    CV mean score     -> -0.5971462847453054
------------------------------
Results for Run ID: e49b45cf9c53456aba1ac32376a51144


## Looking at experimental  results

The results with MLflow’s tracking UI. To start the UI, run the following at the command line:

```bash
mlflow ui
```

Then navigate to http://localhost:5000 in a browser.

## MLflow limitations

1. Relatively new library, thus there are bugs and rough edges.
2. May have too many features for smaller ML projects.
3. No support for collaboration.

## Conclusion


* MLOps is set of best practices to systematically improve machine learning, especially in production environments.
* MLflow is a Python package that supports more mature MLOps workflows.
* MLflow can be used to track machine learning experiments. 